In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [2]:
#import data from housing.csv dataset
df = pd.read_csv('Housing1.csv')
#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('parking', axis=1), df['price'], test_size=0.2)

In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

# Import data from housing.csv dataset
df = pd.read_csv('Housing1.csv')

# Check for and handle missing values
df = df.dropna()  

# Ensure all data is numerical
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# Split data into train and test sets
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: torch.Size([436, 12])
y_train shape: torch.Size([436, 1])
X_test shape: torch.Size([109, 12])
y_test shape: torch.Size([109, 1])


In [15]:
# Define the model
n_features = X_train.shape[1]
model = nn.Sequential(
    nn.Linear(n_features, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [16]:
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

n_epochs = 100   # number of epochs to run
batch_size = 10  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

In [17]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

In [23]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Import data from housing.csv dataset
df = pd.read_csv('Housing1.csv')

# Check for and handle missing values
df = df.dropna()  # or df.fillna(value), depending on your strategy

# Ensure all data is numerical
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# Split data into train and test sets
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

# Print shape of X_train to verify number of features
print("X_train shape:", X_train.shape)

# Define the model
n_features = X_train.shape[1]
model = nn.Sequential(
    nn.Linear(n_features, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

# Loss function and optimizer
loss_fn = nn.MSELoss()  # Mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Increase number of epochs
n_epochs = 1000   # Number of epochs to run
batch_size = 100  # Size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)
# Hold the best model
best_mse = np.inf  # Init to infinity
best_weights = None
history = []

for epoch in range(n_epochs):
    model.train()
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # Take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # Forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            # Update weights
            optimizer.step()
            # Print progress
            bar.set_postfix(mse=float(loss))
    # Evaluate accuracy at end of each epoch
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        mse = loss_fn(y_pred, y_test)
        mse = float(mse)
        history.append(mse)
        if mse < best_mse:
            best_mse = mse
            best_weights = copy.deepcopy(model.state_dict())

# Check if best_weights is None before trying to load state_dict
# Check if best_weights is not None before restoring model
if best_weights is not None:
    # Restore model
    model.load_state_dict(best_weights)
    # Calculate MSE and RMSE
    print("MSE: %.2f" % best_mse)
    print("RMSE: %.2f" % np.sqrt(best_mse))
    # Plot training history
    plt.plot(history)
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.title('Training History')
    plt.show()
else:
    print("No improvement during training. Best MSE:", best_mse)


X_train shape: torch.Size([436, 12])
No improvement during training. Best MSE: inf


In [22]:
# restore model and return best accuracy
model.load_state_dict(best_weights)
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()

TypeError: Expected state_dict to be dict-like, got <class 'NoneType'>.